In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from typing import Union

In [2]:
def get_Van_b():
    """
    Gets the Vandermonde matrix and b we are considering.
    
    """
    n = 15
    m = 100

    alphas = np.array([i/(m - 1) for i in range(m)])
    
    Van = np.ndarray((m, n))
    for i in range(m):
        for j in range(n):
            Van[i, j] = alphas[i]**j
    b = np.zeros(m)
    for i in range(m):
        b[i] = math.exp(math.sin(4 * alphas[i]))
        
    return Van, b

Van, b = get_Van_b()

In [13]:
def back_substitution(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    n = b.size
    x = np.zeros_like(b)

    if A[n-1, n-1] == 0:
        raise ValueError


    x[n-1] = b[n-1]/A[n-1, n-1]
    C = np.zeros((n,n))
    for i in range(n-2, -1, -1):
        bb = 0
        for j in range (i+1, n):
            bb += A[i, j]*x[j]

        C[i, i] = b[i] - bb
        x[i] = C[i, i]/A[i, i]

    return x

def my_qr(A: np.ndarray):
        """
        Returns the QR factorization of a nonsingular array
        via (classical) Gram-Schmidt.

        Parameters
        ----------
        A: np.ndarray
            The matrix with which we find the QR factorization.
            Must be nonsingular, sized n x n

        Returns
        -------
        Q: np.ndarray
            A set of orthonormal column vectors of size n x n

        R: np.ndarray
            An upper triangular matrix
        """
        m, n = A.shape
        Q = np.zeros((m, m))
        R = np.zeros((m, n))

        for i in range(0, n): #Row Iter
            prev = 0 # used to catch r_{jk}q_j in sum
            for j in range(0, i+1): # maintain upper triangularity
                if i != j:
                    R[j, i] = np.dot(Q[:, j], A[:, i])
                    prev += R[j, i] * Q[:, j]
                else: #Diagonal term, take prev
                    R[i, i] = np.linalg.norm(A[:, i] - prev, ord = 2)
                    assert R[i, i] != 0, "Diagonal is zero, function cannot continue"
                    Q[:, j] = (1/R[i, i]) * (A[:, i] - prev)
    
        return (Q, R)

In [4]:
def calculate_Hausholder(v: np.array):
    """
    Helper function to calculate the Hausholder matrix from v.
    
    Parameters
    ----------
    v : np.array
        The given vector. Needs to be normalized prior to passing.
        
    Returns
    -------
    H : np.ndarray
        The Hausholder matrix of v.
    """
    H = np.identity(max(v.shape)) - 2*np.outer(v, v)
    return H

def recover_Qis(A: np.ndarray):
    """
    Calculates all Q_is from a Hausholder applied matrix A.
    
    Parameters
    ----------
    
    A : np.ndarray
        The matrix on which Hausholder QR has been performed
        
    Returns
    -------
    Qis : list(np.ndarray)
        A list containing all Q's 
    """
    Qis = []
    
    m, n = A.shape
    
    for i in range(n):
        v = A[i+1:, i].copy()
        Q = np.identity(m-1)
        Q[i:, i:] = calculate_Hausholder(v)
        Qis.append(Q)
    
    return Qis

def calculate_Q(A: np.ndarray):
    """
    Returns the orthonormal matrix Q from the matrix A after Hausholder QR 
    is applied.
    
    Parameters
    ----------
    
    A : np.ndarray
        The matrix on which Hausholder QR has been performed

    Returns
    -------    
    
    Q: np.ndarray
        The orthonormal matrix involved in QR
    """
    
    Qis = recover_Qis(A.copy())
    Q = np.identity(Qis[0].shape[0])
    for Qi in Qis:
        Q = np.matmul(Q, Qi.T)
        
    return Q


def householder_vectorized(a):
    """Use this version of householder to reproduce the output of np.linalg.qr 
    exactly (specifically, to match the sign convention it uses)
    
    based on https://rosettacode.org/wiki/QR_decomposition#Python
    """
    v = a / (a[0] + np.copysign(np.linalg.norm(a), a[0]))
    v[0] = 1
    tau = 2 / (v.T @ v)
    
    return v,tau

def qr_decomposition(A: np.ndarray) -> Union[np.ndarray, np.ndarray]:
    m,n = A.shape
    R = A.copy()
    Q = np.identity(m)
    
    for j in range(0, n):
        # Apply Householder transformation.
        v, tau = householder_vectorized(R[j:, j, np.newaxis])
        
        H = np.identity(m)
        H[j:, j:] -= tau * (v @ v.T)
        R = H @ R
        Q = H @ Q
        
    return Q[:n].T, np.triu(R[:n])

In [14]:
n = 15
m = 100

#Calcualting QR
Q_HH, R_HH = qr_decomposition(Van)
Q_GS, R_GS = my_qr(Van)

#Take nonzero rows of R. Since Van is full rank, will be n rows
R_HH = np.array([x for x in R_HH if x.any() != 0])

#Check if I did it right
assert R_HH.shape == (n, n), "Oops"

c = np.matmul(Q_HH[:, :n].T, b)

#Solution to part i 
x_soli = back_substitution(R_HH, c)
print("HausHolder Approximate solution to c_14: {} ".format(x_soli[-1]))




#Take nonzero rows of R. Since Van is full rank, will be n rows
R_GS = np.array([x for x in R_GS if x.any() != 0])

#Check if I did it right
assert R_GS.shape == (n, n), "Oops"

c = np.matmul(Q_GS[:, :n].T, b)
x_soli = back_substitution(R_GS, c)
print("Gram Schmidt Approximate solution to c_14: {}".format(x_soli[-1]))

HausHolder Approximate solution to c_14: 2006.7870665592704 
Gram Schmidt Approximate solution to c_14: 1.0996800275618446


In [28]:
test = np.append(Van,b.reshape((b.shape[0],1)) ,1)


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.04121994],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.08406749],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.49580594],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.48201397],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.46916419]])

In [60]:
n = 15
m = 100
aug_A = np.append(Van,b.reshape((b.shape[0],1)) ,1)

#Calcualting QR
Q_HH, R_HH = qr_decomposition(aug_A)
Q_GS, R_GS = my_qr(aug_A)

c = R_HH[:-1, -1]
R_aug = R_HH[:-1, :-1]

x_solii = back_substitution(R_aug, c)
print("HausHolder Approximate solution to c_14: {} ".format(x_solii[-1]))

c = R_GS[:n, -1]
R_aug = R_GS[:n, :-1]
#R_aug = np.array([x for x in R_aug if x.any() != 0])
#c = np.array([x for x in c if x != 0])[:-1]

x_solii = back_substitution(R_aug, c)
print("Gram Schmidt Approximate solution to c_14: {}".format(x_solii[-1]))

HausHolder Approximate solution to c_14: 2006.7870664971297 
Gram Schmidt Approximate solution to c_14: 1.0996800275618466


In [64]:


x_soliii = np.linalg.lstsq(A_n, b_n)
x_soliii

C:\Users\Caleb\AppData\Local\Temp\ipykernel_18660\1155873295.py:4: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  x_soliii = np.linalg.lstsq(A_n, b_n)


(array([ 1.00038598e+00,  3.92939758e+00,  1.03781703e+01, -3.14293464e+01,
         1.71316034e+02, -7.50575741e+02,  1.05210568e+03, -4.21696434e+01,
        -7.27569504e+02, -7.67640457e+01,  4.71729295e+02,  1.88371692e+02,
        -2.64200790e+02, -9.02151071e+01,  8.45629808e+01]),
 array([], dtype=float64),
 12,
 array([1.88115618e+02, 4.46200809e+01, 6.04091055e+00, 6.01275934e-01,
        4.70495215e-02, 2.95724644e-03, 1.50423955e-04, 6.18925596e-06,
        2.04563950e-07, 5.35639315e-09, 1.08606530e-10, 1.64386741e-12,
        1.74658929e-14, 1.06985369e-15, 5.74470067e-16]))

In [73]:
A_n = Van.T @ Van
b_n = Van.T @b

Q_nHH, R_nHH = qr_decomposition(A_n)
Q_nGS, R_nGS = my_qr(A_n)

A_p = np.linalg.inv(R_nHH) @ Q_nHH[:n, :].T

x_soliii = np.linalg.solve(A_p, b_n)
print("HausHolder Approximate solution to c_14: {} ".format(x_soliii[-1]))
A_p = np.linalg.inv(R_nGS) @ Q_nGS[:n, :].T

x_soliii = np.linalg.solve(A_p, b_n)
print("Gram Schmidt Approximate solution to c_14: {}".format(x_soliii[-1]))


HausHolder Approximate solution to c_14: 5004.537912368087 
Gram Schmidt Approximate solution to c_14: -73024.29641227845
